Week 2 

Q1: An insurance company serves ten large customers. The insurance claim placed by each customer in a year is normally distributed with mean 200 and standard deviation 50. (Normal random variables can take negative values but this occurs with small probability when the standard deviation is small compared with the mean and we ignore this possibility) Assume that the insurance claims from different customers are independent. (a) compute the probability that the insurance claim placed by one customer in a year exceeds 250. (b) Compute the probability that the total insurance claim placed by all customers in a year exceeds 2500. 

In [7]:
import numpy as np
import scipy.stats as stats
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_probabilities(mean=200, std_dev=50, threshold=250, total_customers=10, total_threshold=2500):
    """
    Calculate probabilities for insurance claims.

    Parameters:
    mean (float): Mean of the normal distribution for individual claims.
    std_dev (float): Standard deviation of the normal distribution for individual claims.
    threshold (float): Threshold for individual claims.
    total_customers (int): Total number of customers.
    total_threshold (float): Threshold for total claims from all customers.

    Returns:
    tuple: Probability for individual claim exceeding threshold and total claims exceeding total_threshold.
    """
    try:
        # Commonly used values
        norm_dist = stats.norm(mean, std_dev)
        total_mean = total_customers * mean
        total_std_dev = np.sqrt(total_customers) * std_dev
        total_norm_dist = stats.norm(total_mean, total_std_dev)
        
        # (a) Probability that one customer's claim exceeds the threshold
        prob_individual = 1 - norm_dist.cdf(threshold)

        # (b) Probability that total claims from all customers exceed the total_threshold
        prob_total = 1 - total_norm_dist.cdf(total_threshold)

        return prob_individual, prob_total
    
    except Exception as e:
        logging.error(f"An error occurred during calculation: {e}")
        return None

# Example usage
prob_a, prob_b = calculate_probabilities()
prob_a, prob_b

(0.15865525393145707, 0.0007827011290012509)

Q2: An insurance company serves ten large customers. The insurance claim placed by each customer in a year is uniform distributed between 0 and 100. Assume that the insurance claims from different customers are independent. Use the central limit theorem to approximately compute the probability that the total insurance claim placed by all customers in a year exceeds 625. 

In [8]:
import numpy as np
import scipy.stats as stats
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_probability_uniform(a=0, b=100, total_customers=10, total_threshold=625):
    """
    Calculate the probability that the total insurance claim from all customers exceeds a given threshold
    using the central limit theorem for uniformly distributed claims.

    Parameters:
    a (float): Lower bound of the uniform distribution.
    b (float): Upper bound of the uniform distribution.
    total_customers (int): Total number of customers.
    total_threshold (float): Threshold for total claims from all customers.

    Returns:
    float: Probability that the total claims exceed the total_threshold.
    """
    try:
        # Commonly used values
        mean = (a + b) / 2
        variance = ((b - a) ** 2) / 12
        std_dev = np.sqrt(variance)
        
        total_mean = total_customers * mean
        total_std_dev = np.sqrt(total_customers) * std_dev
        total_norm_dist = stats.norm(total_mean, total_std_dev)
        
        # Calculate probability using normal approximation (CLT)
        prob_total = 1 - total_norm_dist.cdf(total_threshold)

        return prob_total
    
    except Exception as e:
        logging.error(f"An error occurred during calculation: {e}")
        return None

# Example usage
prob = calculate_probability_uniform()
prob

0.08545176011539879

Q3: You are trying to determine how many (weekly) magazines to produce in any production run. It costs you 1.00 to print each magazine and you sell magazines for 2.50 each. if you don't sell all your magazines, then the excess must be disposed at a cost of .05 per magazine. Demand per week is D. Assume weekly demandeds are i.i.d. Let x be the number of magazines you decide to produce each week, and let f(x) be the resulting expected profit per week. (a) Build a pandas dataframe that computes, for any given value of x, an estimate of f(x) along with a 95% confidence interval based on demand per week, D. (b) use your pandas dataframe to obtain the approximately optimal value, x*, that maximizes f(x) and report (i) your estimasted confidence interval for f(x*). (c) Say your confidence interval from Part (b) is [a1, a2] for certain values a1 and a2. Is it true that the random profit next week will lie in the interval [a1,a2] with probability .95? Explain

In [9]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def simulate_demand(weeks, mean_demand, std_dev_demand):
    """Simulate weekly demand over a specified number of weeks."""
    return np.random.normal(mean_demand, std_dev_demand, weeks)

def calculate_profit(x, demand, sell_price=2.5, print_cost=1.0, disposal_cost=0.05):
    """Calculate the profit based on the number of magazines produced (x) and actual demand."""
    revenue = np.minimum(x, demand) * sell_price
    production_cost = x * print_cost
    disposal_cost_total = np.maximum(0, x - demand) * disposal_cost
    profit = revenue - production_cost - disposal_cost_total
    return profit

def estimate_expected_profit(x, demands, sell_price=2.5, print_cost=1.0, disposal_cost=0.05):
    """Estimate the expected profit and its 95% confidence interval for a given production quantity (x)."""
    profits = [calculate_profit(x, demand, sell_price, print_cost, disposal_cost) for demand in demands]
    mean_profit = np.mean(profits)
    std_dev_profit = np.std(profits)
    if std_dev_profit == 0:
        # Handle the case where standard deviation is zero
        confidence_interval = (mean_profit, mean_profit)
    else:
        confidence_interval = stats.norm.interval(0.95, loc=mean_profit, scale=std_dev_profit/np.sqrt(len(profits)))
    return mean_profit, confidence_interval

def build_profit_dataframe(mean_demand, std_dev_demand, weeks=1000, max_x=200):
    """Build a dataframe that computes the expected profit and 95% confidence interval for different values of x."""
    try:
        demands = simulate_demand(weeks, mean_demand, std_dev_demand)
        data = {
            'x': [],
            'Expected Profit': [],
            '95% CI Lower Bound': [],
            '95% CI Upper Bound': []
        }

        for x in range(1, max_x + 1):
            mean_profit, confidence_interval = estimate_expected_profit(x, demands)
            data['x'].append(x)
            data['Expected Profit'].append(mean_profit)
            data['95% CI Lower Bound'].append(confidence_interval[0])
            data['95% CI Upper Bound'].append(confidence_interval[1])

        df = pd.DataFrame(data)
        return df
    
    except Exception as e:
        logging.error(f"An error occurred during dataframe construction: {e}")
        return None

def display_optimal_results(df):
    """Display the optimal production quantity and associated results."""
    optimal_row = df.loc[df['Expected Profit'].idxmax()]
    optimal_x = optimal_row['x']
    optimal_expected_profit = optimal_row['Expected Profit']
    optimal_ci_lower = optimal_row['95% CI Lower Bound']
    optimal_ci_upper = optimal_row['95% CI Upper Bound']
    
    print(f"Optimal Production Quantity (x): {optimal_x}")
    print(f"Optimal Expected Profit: {optimal_expected_profit:.2f}")
    print(f"95% Confidence Interval for Expected Profit: [{optimal_ci_lower:.2f}, {optimal_ci_upper:.2f}]")
    return optimal_x, optimal_expected_profit, (optimal_ci_lower, optimal_ci_upper)

def display_selected_results(df, row_start, row_end, col_start, col_end):
    """Display specific results from the dataframe based on user selection."""
    selected_rows = df.iloc[row_start:row_end, col_start:col_end]
    print(selected_rows)
    return selected_rows

# Parameters
mean_demand = 100
std_dev_demand = 20
weeks = 1000
max_x = 200

# Build dataframe
profit_df = build_profit_dataframe(mean_demand, std_dev_demand, weeks, max_x)
if profit_df is not None:
    # Display the optimal results
    optimal_results = display_optimal_results(profit_df)
    
    # Display specific selected results
    selected_results = display_selected_results(profit_df, 143, 144, 2, 3)

Optimal Production Quantity (x): 105.0
Optimal Expected Profit: 131.38
95% Confidence Interval for Expected Profit: [129.36, 133.40]
     95% CI Lower Bound
143          102.154269


Problem Explanation

Imagine you are running a business where you produce and sell magazines every week. You need to figure out how many magazines to produce to make the most profit.

    Costs and Sales:
        It costs you $1.00 to print each magazine.
        You sell each magazine for $2.50.
        If you print too many magazines and don't sell them all, you have to throw away the extra ones, which costs you $0.05 per magazine.

    Weekly Demand:
        The number of magazines people want to buy each week (demand) can change and is different every week.

    Goal:
        You need to decide how many magazines to produce each week (xx) to maximize your profit.
        f(x)f(x) is the expected profit you will make based on the number of magazines you produce.

Steps

    Build a DataFrame:
        Use a computer program to simulate different scenarios of magazine demand.
        Calculate the profit for different numbers of magazines produced.
        Estimate the profit and a confidence interval (range where we are 95% sure the true profit will be) for each number of magazines produced.

    Find the Best Number:
        Determine the number of magazines (xx) that gives you the highest expected profit.
        Report the confidence interval for this profit.

    Interpret the Confidence Interval:
        Explain what the confidence interval means in terms of the expected profit and if it tells us anything about the actual profit we will get next week.